### Atualização da Criação e Execução de Janela para Controle de Estoque

##### Implementações para verificar e validar as inserções dos usuários e novas funcionalidades propostas por mim para um melhor controle de estoque

In [7]:
from tkinter import *
import pyodbc
from datetime import datetime

#Conexão com o Banco de Dados:

dados_conexao = ('Driver={SQLite3 ODBC Driver};'
                 'Server=localhost;'
                 'Database=Dados - Controle de Estoque/Estoque.db;')

conexao = pyodbc.connect(dados_conexao)
cursor = conexao.cursor()

#Funcionalidades do Sistema:

def adicionar_insumo():
    """Adiciona um insumo ao estoque."""
    # Verifica se os campos obrigatórios estão preenchidos
    if not all([nome_insumo.get(), qtde_insumo.get(), data_insumo.get(), lote_insumo.get()]):
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', 'Erro: Preencha todos os campos obrigatórios.')
        return

    # Verifica se a quantidade inserida é um número válido
    try:
        quantidade = int(qtde_insumo.get())
        if quantidade <= 0:
            raise ValueError
    except ValueError:
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', 'Erro: Quantidade inválida. Insira um número inteiro maior que zero.')
        return

    # Verifica se a data inserida é válida e posterior à data atual
    try:
        data_validade = datetime.strptime(data_insumo.get(), '%Y-%m-%d')
        if data_validade.date() < datetime.now().date():
            raise ValueError
    except ValueError:
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', 'Erro: Data de validade inválida ou anterior à data atual.')
        return

    # Verifica se o insumo já está no banco de dados
    cursor.execute(f'''
        SELECT Produto, Quantidade FROM Estoque
        WHERE Produto = "{nome_insumo.get()}" AND DataValidade = "{data_insumo.get()}" AND Lote = "{lote_insumo.get()}"
    ''')
    insumo_existente = cursor.fetchone()
    if insumo_existente:
        produto, qtd_existente = insumo_existente
        nova_quantidade = qtd_existente + quantidade
        cursor.execute(f'''
            UPDATE Estoque
            SET Quantidade = {nova_quantidade} 
            WHERE Produto = "{produto}" AND DataValidade = "{data_insumo.get()}" AND Lote = "{lote_insumo.get()}"
        ''')
        cursor.commit()
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', f'Sucesso: {quantidade} unidade(s) de "{nome_insumo.get()}" adicionado(s) ao estoque.')
    else:
        # Insere o novo insumo no banco de dados
        cursor.execute(f'''
            INSERT INTO Estoque (Produto, Quantidade, DataValidade, Lote)
            VALUES
            ("{nome_insumo.get()}", {quantidade}, "{data_insumo.get()}", {lote_insumo.get()})
        ''')
        cursor.commit()

        caixa_texto.delete('1.0', END)
        # Exibe mensagem de sucesso
        caixa_texto.insert('1.0', f'Sucesso: {quantidade} unidade(s) de "{nome_insumo.get()}" adicionado(s) com sucesso!')
    
    # Limpa os campos de entrada de dados
    nome_insumo.delete('1.0', END)
    qtde_insumo.delete('1.0', END)
    data_insumo.delete('1.0', END)
    lote_insumo.delete('1.0', END)


def deletar_insumo():
    if len(nome_insumo.get()) < 2:
        # exibir uma mensagem -> nome insumo inválido
        # deletar tudo da caixa de texto
        caixa_texto.delete('1.0', END)
        # escrever na caixa de texto
        caixa_texto.insert('1.0', 'Nome do Insumo inválido!')
        # finalizar a função
        return
    
    # deletar o insumo
    cursor.execute(f'''
    DELETE FROM Estoque 
    WHERE Produto="{nome_insumo.get()}"
    ''')
    cursor.commit()
    
    # deletar tudo da caixa de texto
    caixa_texto.delete('1.0', END)
    
    # escrever na caixa de texto
    caixa_texto.insert('1.0', f'{nome_insumo.get()} deletado com sucesso!') 

def consumir_insumo():
    if len(nome_insumo.get()) < 2 or len(lote_insumo.get()) < 1 or len(qtde_insumo.get()) < 1:
        # exibir uma mensagem -> nome insumo inválido
        # deletar tudo da caixa de texto
        caixa_texto.delete('1.0', END)
        # escrever na caixa de texto
        caixa_texto.insert('1.0', f'Nome, Lote e Quantidade do Insumo inválidos!')
        # finalizar a função
        return
    
    # consumir o insumo
    cursor.execute(f'''
    UPDATE Estoque
    SET Quantidade=Quantidade-{qtde_insumo.get()}
    WHERE Produto="{nome_insumo.get()}" AND Lote={lote_insumo.get()}
    ''')
    cursor.commit()
    
    # deletar tudo da caixa de texto
    caixa_texto.delete('1.0', END)
    
    # escrever na caixa de texto
    caixa_texto.insert('1.0', f'{nome_insumo.get()} foi consumido em {qtde_insumo.get()} unidades!')

def visualizar_insumo():
    if len(nome_insumo.get()) < 2:
        # exibir uma mensagem -> nome insumo inválido
        # deletar tudo da caixa de texto
        caixa_texto.delete('1.0', END)
        # escrever na caixa de texto
        caixa_texto.insert('1.0', f'Nome do Insumo inválido!')
        # finalizar a função
        return
    
    # pesquisar pelo insumo
    cursor.execute(f'SELECT * FROM Estoque WHERE Produto="{nome_insumo.get()}"')
    valores = cursor.fetchall()

    texto = ""
    for id_produto, nome, quantidade, validade, lote in valores:
        texto = texto + f'''
        -----
        Produto: {nome}
        Quantidade: {quantidade}
        Validade: {validade}
        Lote: {lote}
        '''
    
    # deletar tudo da caixa de texto
    caixa_texto.delete('1.0', END)
    # escrever na caixa de texto
    caixa_texto.insert('1.0', texto)


#Criação da Janela:
    
window = Tk()

window.geometry('711x646')
window.configure(bg = '#ffffff')
canvas = Canvas(
    window,
    bg = '#ffffff',
    height = 646,
    width = 711,
    bd = 0,
    highlightthickness = 0,
    relief = 'ridge')
canvas.place(x = 0, y = 0)

background_img = PhotoImage(file = 'Janela - Controle de Estoque/background.png')
background = canvas.create_image(
    355.5, 323.0,
    image=background_img)

img0 = PhotoImage(file = 'Janela - Controle de Estoque/img0.png')
b0 = Button(
    image = img0,
    borderwidth = 0,
    highlightthickness = 0,
    command = visualizar_insumo,
    relief = 'flat')

b0.place(
    x = 479, y = 195,
    width = 178,
    height = 38)

img1 = PhotoImage(file = 'Janela - Controle de Estoque/img1.png')
b1 = Button(
    image = img1,
    borderwidth = 0,
    highlightthickness = 0,
    command = deletar_insumo,
    relief = 'flat')

b1.place(
    x = 247, y = 197,
    width = 178,
    height = 36)

img2 = PhotoImage(file = 'Janela - Controle de Estoque/img2.png')
b2 = Button(
    image = img2,
    borderwidth = 0,
    highlightthickness = 0,
    command = consumir_insumo,
    relief = 'flat')

b2.place(
    x = 479, y = 123,
    width = 178,
    height = 35)

img3 = PhotoImage(file = 'Janela - Controle de Estoque/img3.png')
b3 = Button(
    image = img3,
    borderwidth = 0,
    highlightthickness = 0,
    command = adicionar_insumo,
    relief = 'flat')

b3.place(
    x = 247, y = 125,
    width = 178,
    height = 34)

entry0_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox0.png')
entry0_bg = canvas.create_image(
    455.0, 560.0,
    image = entry0_img)

caixa_texto = Text(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

caixa_texto.place(
    x = 250, y = 502,
    width = 410,
    height = 114)

entry1_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox1.png')
entry1_bg = canvas.create_image(
    517.0, 294.5,
    image = entry1_img)

nome_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

nome_insumo.place(
    x = 377, y = 278,
    width = 280,
    height = 31)

entry2_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox2.png')
entry2_bg = canvas.create_image(
    517.0, 340.5,
    image = entry2_img)

data_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

data_insumo.place(
    x = 377, y = 324,
    width = 280,
    height = 31)

entry3_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox3.png')
entry3_bg = canvas.create_image(
    517.0, 388.5,
    image = entry3_img)

lote_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

lote_insumo.place(
    x = 377, y = 372,
    width = 280,
    height = 31)

entry4_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox4.png')
entry4_bg = canvas.create_image(
    517.0, 436.5,
    image = entry4_img)

qtde_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

qtde_insumo.place(
    x = 377, y = 420,
    width = 280,
    height = 31)

window.resizable(False, False)
window.mainloop()

cursor.close()
conexao.close()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\diego\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_13408\1674722507.py", line 75, in adicionar_insumo
    nome_insumo.delete('1.0', END)
  File "d:\Users\diego\anaconda3\lib\tkinter\__init__.py", line 3039, in delete
    self.tk.call(self._w, 'delete', first, last)
_tkinter.TclError: bad entry index "1.0"
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\diego\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_13408\1674722507.py", line 75, in adicionar_insumo
    nome_insumo.delete('1.0', END)
  File "d:\Users\diego\anaconda3\lib\tkinter\__init__.py", line 3039, in delete
    self.tk.call(self._w, 'delete', first, last)
_tkinter.TclError: bad entry index "1.0"
